<a href="https://colab.research.google.com/github/SaswatD27/21cs460_group01/blob/main/Code/Subclustering_MNIST(Not%20Working%20Yet).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import time
import numpy as np
import tensorflow as tf
#from tensorflow_federated import federated_mean
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow import keras
from copy import deepcopy
from sklearn.cluster import DBSCAN

In [ ]:
#(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
#emnist_train, emnist_test = datasets.emnist.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0
img_rows, img_cols = 28, 28

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
K=100
n_k= np.random.randint(100,600,K)
#n_k=[600 for i in range(K)]
n_tot=sum(n_k)
client_images=deepcopy(list(range(len(train_images))))
random.shuffle(client_images)
#print(lenMNIST)
x_train=[]
y_train=[]
'''
for i in range(K):
  x_train.append([])
  y_train.append([])
  for j in range(n_k[i]):
    k=client_images.pop()
    x_train[i].append(train_images[k])
    y_train[i].append(train_labels[k])
'''
indices_dev=[]
for i in range(K):
  indices_dev.append([])
  for j in range(n_k[i]):
    k=client_images.pop()
    indices_dev[i].append(k)  

In [ ]:
print(n_tot)

35853


In [ ]:
#works!
def create_DNN():
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=32, activation='relu'))
  #model.add(tf.keras.layers.Dropout(rate=0.2))
  #model.add(tf.keras.layers.BatchNormalization())
  #model.add(tf.keras.layers.Dense(units=512, activation='relu'))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(units=10, activation='softmax')) 
  #model.add(tf.keras.layers.BatchNormalization())
  return model

In [ ]:
model=create_DNN()
model.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
model.summary()
print(np.array(model.get_weights()))

In [ ]:
def device_local_learning(server_model,id): #Conducts local training for SGD and outputs weight vector
    model=server_model
    model.compile(optimizer='adam',
                  loss="sparse_categorical_crossentropy",
                  metrics=['accuracy'])
    history = model.fit(train_images[indices_dev[id]],train_labels[indices_dev[id]], epochs=20, validation_data=(test_images, test_labels), verbose=0)
    #print(id," done.")
    return model, n_k[id]

In [ ]:
def avg_weights(client_frac,client_models,n_clients_round):
  client_models=np.array(client_models)
  res=client_frac[0]*np.array(client_models[0].get_weights())
  #print(res)
  for i in range(1,n_clients_round):
    res+=client_frac[i]*np.array(client_models[i].get_weights())
  return res

In [ ]:
#FED_AVG_DNN : WORKS NOW
def fed_avg_DNN(server_model, client_models, client_n):
  client_frac=np.zeros(len(client_n))
  for i in range(len(client_n)):
    client_frac[i]=(client_n[i])*(1/np.sum(client_n))
  #print("Sum of client_frac = ",np.sum(client_frac)," client_frac = ",client_frac)
  #client_weights_sum=np.sum(client_models)
  #for layer in server_model.layers:
  #  layer.set_weights([np.dot(client_frac,client_model.get_weights()[0]),np.dot(client_frac,client_model.get_weights()[1])])
  #server_model.set_weights([np.dot(client_frac,[m.get_weights()[0] for m in client_models]),np.dot(client_frac,[m.get_weights()[1] for m in client_models])])
  server_model.set_weights(avg_weights(client_frac,client_models,len(client_frac)))
  return server_model

##Subclustering Code

In [ ]:
def federated_learning_new(T, K, C, server_model):
    w=server_model
    T1 = T
    #w_ = np.zeros((int(C * K), w_n))
    #n_k = np.zeros(int(C * K))
    client_models=[]
    client_n=[]
    # Train for T many rounds
    for i in range(T1):
        training_subset = np.random.randint(0, K, int(C * K))
        z = 0
        for j in training_subset:
          client_models.append(device_local_learning(w, j)[0])
          client_n.append(device_local_learning(w,j)[1])
          z += 1
        w_k = fed_avg_DNN(server_model, client_models, client_n)
        #w = w_k
        print("Global Iteration ",i," done.")
    #print(w_)
    w_global=w_k
    w_k = new_update(client_models, training_subset, client_n, T)
    return w_k,w_global


def new_update(w_, training_subset, n_k, T):
    print(w_)
    w_mult = []
    clustering = DBSCAN(eps=0.5, min_samples=1).fit(np.array([tf.make_ndarray(v).flatten() for v in w_],dtype=object))
    print(clustering.labels_)

    for i in range(np.max(clustering.labels_) + 1):
        training_subset_n = []
        n_k_n = []
        w_new = []
        for k in range(len(training_subset)):
            lab = clustering.labels_[k]
            if i == lab:
                training_subset_n.append(training_subset[k])
                n_k_n.append(n_k[k])
                w_new.append(w_[k])

        wt = fed_avg_DNN(server_model, w_new, n_k_n)
        n_k_t = []
        w_t = []
        for j in range(T):
            z = 0
            for m_t in training_subset_n:
              w_t.append(device_local_learning(wt, m_t)[0])
              n_k_t.append(device_local_learning(wt, m_t)[1])
            w_k_t = fed_avg_DNN(server_model, w_new, n_k_n)
            #wt = w_k_t
            print("Class ", i," Iteration ",j," done.")
        w_mult.append(w_k_t)

    return w_mult

In [ ]:
def average_testing_error(w, testing_x, testing_y):
    error = 0
    for i in range(len(testing_x)):
        y_p = np.dot(w, testing_x[i])
        y_a = testing_y[i]
        # print(i)
        error += (y_a - y_p) ** 2
    error *= (1 / len(testing_x))
    return error

In [ ]:
server_model=create_DNN()

In [ ]:
T = 1  # No of federated learning rounds
K = 10  # Total no of nodes
C = 0.5  # fraction of nodes used at each iteration
# B = #Local batch size used at each learning iteration
# random Model
# x_cord1, y_cord1 = get_data(m, n_k)

time1 = time.time()
w1,w_global = federated_learning_new(T, K, C, server_model)
time2 = time.time()
print("Time taken for SGD federated learning", time2 - time1)

Global Iteration  0  done.
[<keras.engine.sequential.Sequential object at 0x7fbc404dc6d0>, <keras.engine.sequential.Sequential object at 0x7fbc404dc6d0>, <keras.engine.sequential.Sequential object at 0x7fbc404dc6d0>, <keras.engine.sequential.Sequential object at 0x7fbc404dc6d0>, <keras.engine.sequential.Sequential object at 0x7fbc404dc6d0>]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  


AttributeError: ignored